In [ ]:
import cv2
import os
import re
import sys

from PIL import Image
from PIL.Image import core as _imaging

#sys.path.append('C:\WinPython-64bit-2.7.10.3\python-2.7.10.amd64\Lib\site-packages\pil-1.1.7-py2.7-win-amd64.egg')
#import _imaging
#import Image

from facerec.feature import Fisherfaces
from facerec.distance import EuclideanDistance
from facerec.classifier import NearestNeighbor
from facerec.model import PredictableModel
from facerec.validation import KFoldCrossValidation
from facerec.visual import subplot
from facerec.util import minmax_normalize

import numpy as np

In [ ]:
class FisherFaceRec:
    
    def __init__ (self,datafolder, imagefolder, data=[]):
        self._data_, self._X_, self._y_ = [], [], []
        logfiles = [f for f in os.listdir(datafolder) if re.search(r'fold_[0-9]+_data.txt$', f)]
        
        if len(data) <= 0:
            for each in logfiles:

                fp = open(os.path.join(datafolder,each))
                fp.readline()
                for line in fp:
                    row = re.sub(r'(.*)\(([0-9]+), ([0-9]+)\)(.*)$',r'\1(\2,\3)\4',line).split()
                    who = [f for f in os.listdir(os.path.join(imagefolder,row[0])) if re.search(re.compile(r'.*\.'+row[2]+'\.'+row[1]+r'$'), f)]
                    if len(who) == 1:
                        self._data_.append({os.path.join(imagefolder,row[0],who[0]):{'subject_id':row[2],'age_range':row[3],'gender':row[4]}})
                    else:
                        print(who)

                fp.close()
        else:
            self._data_ = data
            
    def __del__ (self):
        del self._data_

    def GetTraining (self,feature):
        for item in self._data_:
            try:
                im = Image.open(item.keys()[0])
                im = im.convert('L')
                self._X_.append(np.asarray(im,dtype=np.uint8))
                self._y_.append(item.values()[0][feature])
            except IOError as e:
                print "I/O error: ",str(e)
            except:
                print "Unexpected error:", sys.exc_info()[0]
                raise
                
        return X, y
    
    def GetModel (self,feature,classifier=NearestNeighbor(dist_metric=EuclideanDistance(), k=1)):
        return PredictableModel(feature=Fisherfaces(), classifier=classifier).compute(self.GetTraining(feature))

In [ ]:
datafolder = 'data'
imagefolder = os.path.join(datafolder,'aligned')

rec = FisherFaceRec(datafolder,imagefolder,data=rec._data_)

In [ ]:
model = rec.GetModel('gender')
cv = KFoldCrossValidation(model,k=10)
cv.validate(model._X_, model._y_)
print cv